# How to score a submission (locally)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gzip
import json
import csv

In [3]:
### Auxiliary functions to process the data:
def _jl_to_list(f, sample):
    output = []
    for e,line in enumerate(f):
        if sample and sample>e:
            output.append(json.loads(line))
        elif not sample:
            output.append(json.loads(line))
            
    return output

def jl_to_list(fname, sample=False):
    
    if fname.suffix == '.gz':
        with gzip.open(fname, 'r') as f:
            return _jl_to_list(f, sample)
    else:
        with open(fname, 'r') as f:
            return _jl_to_list(f, sample)

def read_csv(filename):
    output = []
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            output.append(row)
    return output

#### #1 Configure the path where your data is stored

In [4]:
from pathlib import Path
path = Path('data')

#### #2 Load the data

In [5]:
item_data = jl_to_list(path/'item_data.jl.gz') # --> You will always need to load the full item_data file, as the domains information will be extracted from here.
train_dataset = jl_to_list(path/'train_dataset.jl.gz')[0:100] ### For this example I will just load the first 100 rows of the dataset

#### #3 Get the targets 

The scoring function will expect a list of item-ids

In [6]:
y_true = [user['item_bought'] for user in train_dataset]

#### #4 Generate your submission

In this case, I will generate random recommendations. The scoring function will expect a list of lists.

In [7]:
import random
y_pred = []
for i in range(len(y_true)):
    random_recom = random.choices(item_data, k=10)
    y_pred.append([recom['item_id'] for recom in random_recom])

#### #5 Score your submission




Pass the targets and the recommendations together with the item data to the ndcg_score function



In [8]:
from challenge_metric import ndcg_score

ndcg_score = ndcg_score(y_true, y_pred, item_data,n_predictions=10)

print(f'Your score is: {ndcg_score}')

Your score is: 0.0005332917917927891
